# [Advent of Code 2021](https://adventofcode.com/2021)

Starting from day 10, I'll try to follow [Norvig's approach](https://github.com/norvig/pytudes/blob/main/ipynb/Advent-2020.ipynb), with all solutions in a single notebook.

# Helpers

Most of the helpers are taken from Norvig's notebook.

In [1]:
import math
import re
import itertools
from collections import Counter, defaultdict, namedtuple, deque
from itertools import permutations, combinations, product
from functools import cache, reduce
from typing import Tuple

def first(iterable, default=None):
    "Return first item in `iterable`, or `default`."
    return next(iter(iterable), default)

def quantify(iterable, pred=bool) -> int:
    "Count the number of items in `iterable` for which `pred` is true."
    return sum(1 for item in iterable if pred(item))

def ints(text: str) -> Tuple[int]:
    "Return a tuple of all the integers in `text`."
    return tuple(map(int, re.findall('-?[0-9]+', text)))

def data(inp: str, parser=str, sep='\n') -> list:
    "Parse input into sections separated by `sep`, and apply `parser` to each."
    sections = inp.rstrip().split(sep)
    return [parser(section) for section in sections]

cat = ''.join
flatten = itertools.chain.from_iterable

# Day 10: Syntax Scoring

Analyze inputs with unmatched characters.

In [2]:
ex10 = '''[({(<(())[]>[[{[]{<()<>>
[(()[<>])]({[<{<<[]>>(
{([(<{}[<>[]}>{[]{[(<()>
(((({<>}<{<{<>}{[]{[]{}
[[<[([]))<([[{}[[()]]]
[{[{({}]{}}([{[{{{}}([]
{<[[]]>}<{[{[{[]{()[[[]
[<(<(<(<{}))><([]([]()
<{([([[(<>()){}]>(<<{{
<{([{{}}[<[[[<>{}]]]>[]]'''
in10 = open('day10.txt').read()

In [3]:
def parse10(I): return data(I)
#parse10(ex10)

In [4]:
def match_errors(line):
    open, close = '([{<', ')]}>'
    s = []
    while line:
        c, *line = line
        if c in close:
            if s.pop() != open[close.index(c)]:
                return close.index(c) # unexpected character index ('corrupted' line)
        elif c in open:
            s.append(c)
    return s # unbalanced characters stack ('incomplete' line)

In [5]:
def day10_1(D):
    points = [3, 57, 1197, 25137]
    return sum(points[r] for r in map(match_errors, D) if not isinstance(r, list))
day10_1(parse10(ex10))

26397

In [6]:
day10_1(parse10(in10))

388713

In [7]:
def day10_2(D):
    def score(cs): return reduce(lambda r, c: r * 5 + '([{<'.index(c) + 1, cs, 0)
    scores = sorted(score(reversed(r)) for r in map(match_errors, D) if isinstance(r, list))
    return scores[len(scores) // 2]
day10_2(parse10(ex10))

288957

In [8]:
day10_2(parse10(in10))

3539961434

# Day 11: Dumbo Octopus

The task is to implement the algorithm that looks like Game of Life, with some differences:
* The grid is fixed size - 10x10. Game of Life has infitite grid.
* Calculation next step can take multiple iteration (in a [fixed-point](https://en.wikipedia.org/wiki/Fixed_point_(mathematics)) manner). Calculating Game of life next generation is much simpler.

In [9]:
ex11 = '''5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526'''
in11 = open('day11.txt').read()

In [10]:
def parse11(I): return data(I, lambda a: list(map(int, a)))
#parse11(ex11)

Here is the core of the solution - `octopus_flashes`. It generates numbers of flashes for each step.

Current grid is tracked in `g` dictionary. It maps *coordinate* to an octopus *energy value*.

Auxilary data structures:

* `step_flashes` - set of octopuses coordinates that flashed during the current step *so far*
* `flashes` - set of octopus coordinates that flashed *right now* 
* `adds` - mapping *coordinate* to the current flash-induced *energy increase*. I use [`collections.Counter`](https://docs.python.org/3/library/collections.html#collections.Counter) to count how many times neighbouring octopuses flashed and subsequently causes energy increase for the current octopus.

In [11]:
def octopus_flashes(I, steps):
    g = {(x,y): c for y, r in enumerate(I) for x, c in enumerate(r)}
    for _ in range(steps):
        g = {p: c+1 for p, c in g.items()}
        step_flashes = set()
        while flashes := {p for p, c in g.items() if c > 9}:
            step_flashes |= flashes
            adds = Counter((x+dx, y+dy) for (x, y) in flashes for dx in (-1,0,1) for dy in (-1,0,1))
            g = {p: 0 if p in step_flashes else c+adds[p] for p, c in g.items()}
        yield len(step_flashes)

In [12]:
def day11_1(D): return sum(octopus_flashes(D, 100))
day11_1(parse11(ex11))

1656

In [13]:
day11_1(parse11(in11))

1608

In [14]:
def day11_2(D): return next(i+1 for i, f in enumerate(octopus_flashes(D, 10**6)) if f == 100)
day11_2(parse11(ex11))

195

In [15]:
day11_2(parse11(in11))

214